In [1]:
import pandas as pd

# Load raw world cities
cities_df = pd.read_csv("/Users/johngrier/CAGE_Strategy/Data/worldcities.csv")

# Filter for national capitals only
capital_df = cities_df[cities_df['capital'] == 'primary'].copy()

# Keep relevant columns
capital_df = capital_df[['country', 'city', 'lat', 'lng']]
capital_df.columns = ['Country', 'Capital', 'Latitude', 'Longitude']

# Rename countries to match other files if needed
capital_df['Country'] = capital_df['Country'].replace({
    'United Kingdom': 'UK',
    'South Korea': 'Korea, South',
    'Russia': 'Russian Federation',
    'Czechia': 'Czech Republic'
})

# Select ~20 countries
countries_needed = [
    'UK', 'Germany', 'Belgium', 'United States', 'Nigeria', 'India',
    'Brazil', 'Mexico', 'South Africa', 'France',
    'Italy', 'Spain', 'China', 'Japan', 'Korea, South',
    'Australia', 'Canada', 'Netherlands', 'Argentina', 'Turkey'
]

# Filter
capital_subset = capital_df[capital_df['Country'].isin(countries_needed)]

# Save cleaned dataset for reuse
capital_subset.to_csv("/Users/johngrier/CAGE_Strategy/Data/capital_coords.csv", index=False)

# Confirm preview
display(capital_subset)

,Country,Capital,Latitude,Longitude
0,Japan,Tokyo,35.6897,139.6922
8,"Korea, South",Seoul,37.5600,126.9900
9,Mexico,Mexico City,19.4333,-99.1333
13,China,Beijing,39.9067,116.3975
18,Argentina,Buenos Aires,-34.6033,-58.3817
35,UK,London,51.5072,-0.1275
36,France,Paris,48.8567,2.3522
84,Spain,Madrid,40.4169,-3.7033
110,Turkey,Ankara,39.9300,32.8500
123,United States,Washington,38.9047,-77.0163


In [3]:
import pandas as pd
from geopy.distance import great_circle

# Load cleaned capital coordinates
capital_df = pd.read_csv("/Users/johngrier/CAGE_Strategy/Data/capital_coords.csv")

# Set UK as the home base
home_country = 'UK'
home_coords = capital_df[capital_df['Country'] == home_country][['Latitude', 'Longitude']].values[0]

# Create distance table
geo_distances = []

for _, row in capital_df.iterrows():
    target_country = row['Country']
    target_coords = (row['Latitude'], row['Longitude'])
    
    # Skip if it's the same country (UK to UK = 0)
    if target_country == home_country:
        continue
    
    distance_km = great_circle(home_coords, target_coords).km
    geo_distances.append({
        'HomeCountry': home_country,
        'TargetCountry': target_country,
        'GeographicDistanceKM': distance_km
    })

# Convert to DataFrame
geo_df = pd.DataFrame(geo_distances)

# Normalize distances for scoring [0, 1]
geo_df['Normalized_GeographicDistance'] = (
    (geo_df['GeographicDistanceKM'] - geo_df['GeographicDistanceKM'].min()) /
    (geo_df['GeographicDistanceKM'].max() - geo_df['GeographicDistanceKM'].min())
)

# Save to CSV for later reuse
geo_df.to_csv("/Users/johngrier/CAGE_Strategy/Data/geographic_distances.csv", index=False)

# Preview
display(geo_df.sort_values(by="GeographicDistanceKM"))

,HomeCountry,TargetCountry,GeographicDistanceKM,Normalized_GeographicDistance
18,UK,Netherlands,311.705620,0.000000
15,UK,Belgium,320.764635,0.000543
5,UK,France,343.517063,0.001908
14,UK,Netherlands,357.317637,0.002736
9,UK,Germany,931.555162,0.037182
6,UK,Spain,1263.378019,0.057087
13,UK,Italy,1433.904881,0.067316
7,UK,Turkey,2832.935325,0.151238
11,UK,Nigeria,4770.072441,0.267439
16,UK,Canada,5360.669021,0.302866


In [7]:
# Create or reload administrative distance data
admin_df = pd.DataFrame([
    {'TargetCountry': 'Germany', 'Colonial': 0, 'TradeBloc': 1, 'LegalSystem': 1, 'Language': 0},
    {'TargetCountry': 'Belgium', 'Colonial': 0, 'TradeBloc': 1, 'LegalSystem': 1, 'Language': 0},
    {'TargetCountry': 'United States', 'Colonial': 1, 'TradeBloc': 0, 'LegalSystem': 1, 'Language': 1},
    {'TargetCountry': 'Nigeria', 'Colonial': 1, 'TradeBloc': 0, 'LegalSystem': 1, 'Language': 1},
    {'TargetCountry': 'India', 'Colonial': 1, 'TradeBloc': 0, 'LegalSystem': 1, 'Language': 1},
    {'TargetCountry': 'France', 'Colonial': 0, 'TradeBloc': 1, 'LegalSystem': 0, 'Language': 0},
    {'TargetCountry': 'Italy', 'Colonial': 0, 'TradeBloc': 1, 'LegalSystem': 0, 'Language': 0},
    {'TargetCountry': 'Spain', 'Colonial': 0, 'TradeBloc': 1, 'LegalSystem': 0, 'Language': 0},
    {'TargetCountry': 'Canada', 'Colonial': 1, 'TradeBloc': 0, 'LegalSystem': 1, 'Language': 1},
    {'TargetCountry': 'Australia', 'Colonial': 1, 'TradeBloc': 0, 'LegalSystem': 1, 'Language': 1},
    {'TargetCountry': 'South Africa', 'Colonial': 1, 'TradeBloc': 0, 'LegalSystem': 1, 'Language': 1},
    {'TargetCountry': 'Brazil', 'Colonial': 0, 'TradeBloc': 0, 'LegalSystem': 0, 'Language': 0},
    {'TargetCountry': 'Mexico', 'Colonial': 0, 'TradeBloc': 0, 'LegalSystem': 1, 'Language': 0},
    {'TargetCountry': 'Japan', 'Colonial': 0, 'TradeBloc': 0, 'LegalSystem': 0, 'Language': 0},
    {'TargetCountry': 'Korea, South', 'Colonial': 0, 'TradeBloc': 0, 'LegalSystem': 1, 'Language': 0},
    {'TargetCountry': 'Netherlands', 'Colonial': 0, 'TradeBloc': 1, 'LegalSystem': 1, 'Language': 0},
    {'TargetCountry': 'Turkey', 'Colonial': 0, 'TradeBloc': 0, 'LegalSystem': 0, 'Language': 0},
    {'TargetCountry': 'Argentina', 'Colonial': 0, 'TradeBloc': 0, 'LegalSystem': 0, 'Language': 0},
    {'TargetCountry': 'China', 'Colonial': 0, 'TradeBloc': 0, 'LegalSystem': 0, 'Language': 0}
])

# Calculate total shared attributes
admin_df["MatchScore"] = admin_df[['Colonial', 'TradeBloc', 'LegalSystem', 'Language']].sum(axis=1)

# Map to AdministrativeDistance scale
admin_df["AdministrativeDistance"] = admin_df["MatchScore"].apply(
    lambda x: 0.0 if x >= 3 else 0.25 if x == 2 else 0.5 if x == 1 else 1.0
)

# Save for reuse
admin_df.to_csv("/Users/johngrier/CAGE_Strategy/Data/administrative_distances.csv", index=False)

In [10]:
admin_df["MatchScore"] = admin_df[['Colonial', 'TradeBloc', 'LegalSystem', 'Language']].sum(axis=1)
admin_df["AdministrativeDistance"] = admin_df["MatchScore"].apply(
    lambda x: 0.0 if x >= 3 else 0.25 if x == 2 else 0.5 if x == 1 else 1.0
)

In [11]:
merged_df = geo_df.merge(admin_df[["TargetCountry", "AdministrativeDistance"]], on="TargetCountry", how="left")
print(merged_df[["TargetCountry", "AdministrativeDistance"]])

    TargetCountry  AdministrativeDistance
0           Japan                    1.00
1    Korea, South                    0.50
2          Mexico                    0.50
3           China                    1.00
4       Argentina                    1.00
5          France                    0.50
6           Spain                    0.50
7          Turkey                    1.00
8   United States                    0.00
9         Germany                    0.25
10   South Africa                    0.00
11        Nigeria                    0.00
12         Brazil                    1.00
13          Italy                    0.50
14    Netherlands                    0.25
15        Belgium                    0.25
16         Canada                    0.00
17   South Africa                    0.00
18    Netherlands                    0.25
19      Australia                    0.00
20   South Africa                    0.00
21          India                    0.00


In [12]:
# Apply consistent renaming
rename_map = {
    'South Korea': 'Korea, South',
    'United Kingdom': 'UK',
    'United States of America': 'United States'
}

geo_df["TargetCountry"] = geo_df["TargetCountry"].replace(rename_map)
admin_df["TargetCountry"] = admin_df["TargetCountry"].replace(rename_map)

In [13]:
geo_df = geo_df.merge(admin_df[["TargetCountry", "AdministrativeDistance"]], on="TargetCountry", how="left")
geo_df["Normalized_AdministrativeDistance"] = geo_df["AdministrativeDistance"].fillna(1.0)  # fallback if merge failed

In [17]:
import pandas as pd

# Load World Bank GDP dataset
gdp_raw = pd.read_csv("/Users/johngrier/CAGE_Strategy/Data/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_9.csv", skiprows=4)

# Select latest available year (e.g., 2022)
latest_year = '2022'
gdp_df = gdp_raw[['Country Name', latest_year]].rename(columns={
    'Country Name': 'Country',
    latest_year: 'GDP_per_capita_USD'
}).dropna()

# Preview
display(gdp_df.head())

,Country,GDP_per_capita_USD
0,Aruba,30559.533535
1,Africa Eastern and Southern,1628.024526
2,Afghanistan,357.261153
3,Africa Western and Central,1777.235012
4,Angola,2929.694455


In [19]:
import numpy as np 

# Renaming for matching
country_rename_map = {
    'United Kingdom': 'UK',
    'United States': 'United States',
    'South Korea': 'Korea, South',
    'Russian Federation': 'Russia'
}

gdp_df["Country"] = gdp_df["Country"].replace(country_rename_map)

# Filter to only countries in geo_df
econ_df = gdp_df[gdp_df["Country"].isin(geo_df["TargetCountry"])].copy()

# Add UK as base
uk_gdp = gdp_df[gdp_df["Country"] == "UK"]["GDP_per_capita_USD"].values[0]

# Compute log-ratio of GDP per capita (Economic Distance)
econ_df["EconomicDistance"] = econ_df["GDP_per_capita_USD"].apply(lambda x: abs(np.log(x / uk_gdp)))

# Normalize to 0–1
econ_df["Normalized_EconomicDistance"] = (
    (econ_df["EconomicDistance"] - econ_df["EconomicDistance"].min()) /
    (econ_df["EconomicDistance"].max() - econ_df["EconomicDistance"].min())
)

# Merge with main CAGE table
geo_df = geo_df.merge(econ_df[["Country", "EconomicDistance", "Normalized_EconomicDistance"]],
                      left_on="TargetCountry", right_on="Country", how="left").drop(columns=["Country"])

In [25]:
country_rename_map = {
    'United Kingdom': 'UK',                 # already matched
    'United States': 'United States',
    'Korea, Rep.': 'Korea, South',          # South Korea
    'Turkiye': 'Turkey',                    # modern Turkish spelling
    'Russian Federation': 'Russia'          # if used
}

In [26]:
# Apply renaming to GDP dataset
gdp_df["Country"] = gdp_df["Country"].replace(country_rename_map)

# Now rebuild filtered GDP subset
econ_df = gdp_df[gdp_df["Country"].isin(geo_df["TargetCountry"])].copy()

# Compute economic distance using log-ratio from UK
uk_gdp = gdp_df[gdp_df["Country"] == "UK"]["GDP_per_capita_USD"].values[0]

econ_df["EconomicDistance"] = econ_df["GDP_per_capita_USD"].apply(lambda x: abs(np.log(x / uk_gdp)))

econ_df["Normalized_EconomicDistance"] = (
    (econ_df["EconomicDistance"] - econ_df["EconomicDistance"].min()) /
    (econ_df["EconomicDistance"].max() - econ_df["EconomicDistance"].min())
)

In [31]:
# Final weights 
weights = {
    "Cultural": 0.25,
    "Administrative": 0.25,
    "Geographic": 0.25,
    "Economic": 0.25
}

# Drop existing EconomicDistance columns to prevent conflict during merge
geo_df = geo_df.drop(columns=["EconomicDistance", "Normalized_EconomicDistance"], errors="ignore")

# Final CAGE score calculation
geo_df = geo_df.merge(
    econ_df[["Country", "EconomicDistance", "Normalized_EconomicDistance"]],
    left_on="TargetCountry", right_on="Country", how="left"
).drop(columns=["Country"])

# Final CAGE score calculation
geo_df["CAGE_Score"] = (
    weights["Cultural"] * geo_df["Normalized_CulturalDistance"] +
    weights["Administrative"] * geo_df["Normalized_AdministrativeDistance"] +
    weights["Geographic"] * geo_df["Normalized_GeographicDistance"] +
    weights["Economic"] * geo_df["Normalized_EconomicDistance"]
)

# Save final results
geo_df.to_csv("/Users/johngrier/CAGE_Strategy/Data/full_cage_scores.csv", index=False)

# Preview
display(geo_df.sort_values("CAGE_Score")[[
    "TargetCountry",
    "Normalized_CulturalDistance",
    "Normalized_AdministrativeDistance",
    "Normalized_GeographicDistance",
    "Normalized_EconomicDistance",
    "CAGE_Score"
]])

,TargetCountry,Normalized_CulturalDistance,Normalized_AdministrativeDistance,Normalized_GeographicDistance,Normalized_EconomicDistance,CAGE_Score
0,Japan,NaN,1.00,0.554690,0.074258,NaN
1,"Korea, South",NaN,0.50,0.512688,0.090613,NaN
2,Mexico,NaN,0.50,0.516898,0.440537,NaN
3,China,NaN,1.00,0.469608,0.404958,NaN
4,Argentina,NaN,1.00,0.648811,0.372899,NaN
5,France,NaN,0.50,0.001908,0.011103,NaN
6,Spain,NaN,0.50,0.057087,0.113122,NaN
7,Turkey,NaN,1.00,0.151238,0.462113,NaN
8,United States,NaN,0.00,0.335007,0.151069,NaN
9,Germany,NaN,0.25,0.037182,0.000000,NaN


In [32]:
missing_culture = geo_df[geo_df["Normalized_CulturalDistance"].isna()]
display(missing_culture[["TargetCountry", "Normalized_CulturalDistance"]])


,TargetCountry,Normalized_CulturalDistance
0,Japan,NaN
1,"Korea, South",NaN
2,Mexico,NaN
3,China,NaN
4,Argentina,NaN
5,France,NaN
6,Spain,NaN
7,Turkey,NaN
8,United States,NaN
9,Germany,NaN


In [34]:
hofstede_fill = [
    # Country         PDI  IDV  MAS  UAI  LTO  IVR
    ["Argentina",      49,  46,  56,  86,  20,  62],
    ["Australia",      36,  90,  61,  51,  21,  71],
    ["Brazil",         69,  38,  49,  76,  44,  59],
    ["Canada",         39,  80,  52,  48,  36,  68],
    ["China",          80,  20,  66,  30,  87,  24],
    ["France",         68,  71,  43,  86,  63,  48],
    ["India",          77,  48,  56,  40,  51,  26],
    ["Italy",          50,  76,  70,  75,  61,  30],
    ["Japan",          54,  46,  95,  92,  88,  42],
    ["Korea, South",   60,  18,  39,  85,  100, 29],
    ["Mexico",         81,  30,  69,  82,  24,  97],
    ["Netherlands",    38,  80,  14,  53,  67,  68],
    ["Nigeria",        80,  30,  60,  55,  13,  84],
    ["South Africa",   49,  65,  63,  49,  34,  63],
    ["Spain",          57,  51,  42,  86,  48,  44],
    ["Turkey",         66,  37,  45,  85,  46,  49],
]

# Convert to DataFrame and append existing csv
hofstede_fill_df = pd.DataFrame(hofstede_fill, columns=[
    "Country", "PDI", "IDV", "MAS", "UAI", "LTO", "IVR"
])

# Append and remove any duplicates
hofstede_df = pd.concat([hofstede_df, hofstede_fill_df], ignore_index=True).drop_duplicates(subset="Country")

# Save to file for future use
hofstede_df.to_csv("/Users/johngrier/CAGE_Strategy/Data/hofstede_scores.csv", index=False)

In [38]:
# Recalculate CulturalDistance using the expanded Hofstede table
home_vec = hofstede_df[hofstede_df["Country"] == "UK"].iloc[:, 1:].values[0]

def calc_cultural_distance(target_country):
    try:
        target_vec = hofstede_df[hofstede_df["Country"] == target_country].iloc[:, 1:].values[0]
        return np.linalg.norm(home_vec - target_vec)
    except:
        return np.nan

geo_df["CulturalDistance"] = geo_df["TargetCountry"].apply(calc_cultural_distance)

# Normalize
geo_df["Normalized_CulturalDistance"] = (
    (geo_df["CulturalDistance"] - geo_df["CulturalDistance"].min()) /
    (geo_df["CulturalDistance"].max() - geo_df["CulturalDistance"].min())
)

# Recompute CAGE score
geo_df["CAGE_Score"] = (
    weights["Cultural"] * geo_df["Normalized_CulturalDistance"] +
    weights["Administrative"] * geo_df["Normalized_AdministrativeDistance"] +
    weights["Geographic"] * geo_df["Normalized_GeographicDistance"] +
    weights["Economic"] * geo_df["Normalized_EconomicDistance"]
)

# Save the updated CAGE scores to CSV
geo_df.to_csv("/Users/johngrier/CAGE_Strategy/Data/full_cage_scores.csv", index=False)

print("Updated full_cage_scores.csv saved with cultural distances and CAGE scores.")

Updated full_cage_scores.csv saved with cultural distances and CAGE scores.
